<h2>Applied DS Capstone Project</h2>
<p>This notebook is the Capstone Project for the Applied Data Science Certification path on Coursera.org</p>

In [1]:
import pandas as pd
import numpy as np
print('hello capstone project course!'.title())

Hello Capstone Project Course!


<h2>Questions 1 through 4</h2>

In [2]:
#Use pandas to read the link provided into a list of dataframes
provided = pd.read_html('http://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')

In [3]:
#find the dataframe for the table of postal codes
for i, tbl in enumerate(provided):
    print(i)
    print(tbl.columns)

0
Index(['Postal Code', 'Borough', 'Neighbourhood'], dtype='object')
1
Int64Index([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17], dtype='int64')
2
Int64Index([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17], dtype='int64')


In [5]:
#create dataframe from scraped dfs and filter out Boroughs containing 'Not assigned'; reset the index
df_postal = provided[0][provided[0]['Borough']!='Not assigned'].sort_values('Postal Code')
df_postal.columns = ['PostalCode', 'Borough', 'Neighborhood']
df_postal

,PostalCode,Borough,Neighborhood
9,M1B,Scarborough,"Malvern, Rouge"
18,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
27,M1E,Scarborough,"Guildwood, Morningside, West Hill"
36,M1G,Scarborough,Woburn
45,M1H,Scarborough,Cedarbrae
...,...,...,...
98,M9N,York,Weston
107,M9P,Etobicoke,Westmount
116,M9R,Etobicoke,"Kingsview Village, St. Phillips, Martin Grove ..."
143,M9V,Etobicoke,"South Steeles, Silverstone, Humbergate, Jamest..."


In [6]:
#use .shape property to get number of rows, columns in filtered df
df_postal.shape

(103, 3)

<h2>End Questions 1 - 4</h2><br><br>
<h2>Begin Geocoded DataFrame</h2>

In [31]:
import json

from geopy.geocoders import Nominatim
from sklearn.cluster import KMeans
from pandas.io.json import json_normalize
import requests
import folium as fm
import matplotlib.cm as cm
import matplotlib.colors as colors

In [8]:
coords = pd.read_csv('Geospatial_Coordinates.csv')
coords.head(10)

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
5,M1J,43.744734,-79.239476
6,M1K,43.727929,-79.262029
7,M1L,43.711112,-79.284577
8,M1M,43.716316,-79.239476
9,M1N,43.692657,-79.264848


In [9]:
collected_df = df_postal.reset_index(drop=True)

In [10]:
collected_df['Lat'] = coords['Latitude']
collected_df['Long'] = coords['Longitude']
collected_df.head(10)

,PostalCode,Borough,Neighborhood,Lat,Long
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"Kennedy Park, Ionview, East Birchmount Park",43.727929,-79.262029
7,M1L,Scarborough,"Golden Mile, Clairlea, Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffside, Cliffcrest, Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848


Check against the image provided by selecting all the same Postal Codes used in the image

In [11]:
checklist = ['M5G', 'M2H', 'M4B', 'M1J', 'M4G', 'M4M', 'M1R','M9V','M9L','M5V','M1B','M5A']

In [12]:
collected_df[collected_df['PostalCode'].isin(checklist)]

,PostalCode,Borough,Neighborhood,Lat,Long
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
11,M1R,Scarborough,"Wexford, Maryvale",43.750072,-79.295849
17,M2H,North York,Hillcrest Village,43.803762,-79.363452
35,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937
38,M4G,East York,Leaside,43.709060,-79.363452
43,M4M,East Toronto,Studio District,43.659526,-79.340923
53,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
57,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383
68,M5V,Downtown Toronto,"CN Tower, King and Spadina, Railway Lands, Har...",43.628947,-79.394420


In [13]:
collected_df.groupby('Borough').count().sort_values('PostalCode').tail()

,PostalCode,Neighborhood,Lat,Long
Borough,,,,
Central Toronto,9,9,9,9
Etobicoke,12,12,12,12
Scarborough,17,17,17,17
Downtown Toronto,19,19,19,19
North York,24,24,24,24


In [14]:
torontos = [tor for tor in collected_df['Borough'].unique() if 'Toronto' in tor]
torontos

['East Toronto', 'Central Toronto', 'Downtown Toronto', 'West Toronto']

In [15]:
address = 'Toronto, ON'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
t_lat = location.latitude
t_long = location.longitude
print(f'The geograpical coordinates of {address} are {t_lat:10f}, {t_long:10f}.')

The geograpical coordinates of Toronto, ON are  43.653482, -79.383935.


In [16]:
t_map = fm.Map(location = [t_lat, t_long], zoom_start=10)

for row in collected_df.iterrows():
    hood = row[1] #borough = [0], hood = [2], lat = [3], long = [4]
    
    label = f'{hood[0]}, {hood[1]}' #Code, borough
    fm.CircleMarker(
        [hood[3], hood[4]], #LAT, LONG
        radius = 5,
        popup = label,
        color = 'blue',
        fill=True,
        fill_color = '#3186cc',
        fill_opacity = .7).add_to(t_map)
    
t_map

In [17]:
df_torontos = collected_df[collected_df['Borough'].isin(torontos)]
df_torontos = df_torontos.sort_values(['Borough','Neighborhood']).reset_index(drop=True)

df_torontos

,PostalCode,Borough,Neighborhood,Lat,Long
0,M4S,Central Toronto,Davisville,43.704324,-79.388790
1,M4P,Central Toronto,Davisville North,43.712751,-79.390197
2,M5P,Central Toronto,"Forest Hill North & West, Forest Hill Road Park",43.696948,-79.411307
3,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790
4,M4T,Central Toronto,"Moore Park, Summerhill East",43.689574,-79.383160
5,M4R,Central Toronto,"North Toronto West, Lawrence Park",43.715383,-79.405678
6,M5N,Central Toronto,Roselawn,43.711695,-79.416936
7,M4V,Central Toronto,"Summerhill West, Rathnelly, South Hill, Forest...",43.686412,-79.400049
8,M5R,Central Toronto,"The Annex, North Midtown, Yorkville",43.672710,-79.405678
9,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306


In [20]:
df_downtown = df_torontos[df_torontos['Borough']=='Downtown Toronto'].reset_index(drop=True)
df_downtown

,PostalCode,Borough,Neighborhood,Lat,Long
0,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306
1,M5V,Downtown Toronto,"CN Tower, King and Spadina, Railway Lands, Har...",43.628947,-79.394420
2,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383
3,M6G,Downtown Toronto,Christie,43.669542,-79.422564
4,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
5,M5L,Downtown Toronto,"Commerce Court, Victoria Hotel",43.648198,-79.379817
6,M5X,Downtown Toronto,"First Canadian Place, Underground city",43.648429,-79.382280
7,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
8,M5J,Downtown Toronto,"Harbourfront East, Union Station, Toronto Islands",43.640816,-79.381752
9,M5T,Downtown Toronto,"Kensington Market, Chinatown, Grange Park",43.653206,-79.400049


In [23]:
downtown_map = fm.Map(location = [t_lat, t_long], zoom_start=12)

for row in df_downtown.iterrows():
    hood = row[1] #borough = [0], hood = [2], lat = [3], long = [4]
    
    label = f'{hood[0]}, {hood[1]}' #Code, borough
    fm.CircleMarker(
        [hood[3], hood[4]], #LAT, LONG
        radius = 5,
        popup = label,
        color = 'blue',
        fill=True,
        fill_color = '#3186cc',
        fill_opacity = .7).add_to(downtown_map)
    
downtown_map

In [101]:
CLIENT_ID = '' # your Foursquare ID
CLIENT_SECRET = '' # your Foursquare Secret
VERSION = '20200828' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)
radius = 500
LIMIT = 100

Your credentails:
CLIENT_ID: 
CLIENT_SECRET:


In [82]:
test_lat, test_long = df_downtown.loc[0,'Lat':'Long']

In [83]:
url = 'https://api.foursquare.com/v2/venues/explore?client_id\
={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, 
                                                            CLIENT_SECRET, 
                                                          test_lat, 
                                                          test_long, 
                                                          VERSION,
                                                          radius, 
                                                          LIMIT)


In [ ]:
results = requests.get(url).json()
results

In [85]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [86]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,LCBO,Liquor Store,43.642944,-79.372440
1,The Keg Steakhouse + Bar - Esplanade,Restaurant,43.646712,-79.374768
2,Fresh On Front,Vegetarian / Vegan Restaurant,43.647815,-79.374453
3,Meridian Hall,Concert Hall,43.646292,-79.376022
4,Biff's Bistro,French Restaurant,43.647085,-79.376342


In [87]:
def getNearbyVenues(names, latitudes, longitudes):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
#         print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius=500&limit=100'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng)
            
        # make the GET request
        results = requests.get(url).json()['response']['groups'][0]['items']
#         print(results)
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [88]:
downtown_venues = getNearbyVenues(names=df_downtown['Neighborhood'],
                                 latitudes=df_downtown['Lat'],
                                 longitudes=df_downtown['Long'])

In [89]:
downtown_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Berczy Park,43.644771,-79.373306,LCBO,43.642944,-79.372440,Liquor Store
1,Berczy Park,43.644771,-79.373306,The Keg Steakhouse + Bar - Esplanade,43.646712,-79.374768,Restaurant
2,Berczy Park,43.644771,-79.373306,Fresh On Front,43.647815,-79.374453,Vegetarian / Vegan Restaurant
3,Berczy Park,43.644771,-79.373306,Meridian Hall,43.646292,-79.376022,Concert Hall
4,Berczy Park,43.644771,-79.373306,Biff's Bistro,43.647085,-79.376342,French Restaurant


In [90]:
downtown_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Berczy Park,57,57,57,57,57,57
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",17,17,17,17,17,17
Central Bay Street,64,64,64,64,64,64
Christie,18,18,18,18,18,18
Church and Wellesley,76,76,76,76,76,76
"Commerce Court, Victoria Hotel",100,100,100,100,100,100
"First Canadian Place, Underground city",100,100,100,100,100,100
"Garden District, Ryerson",100,100,100,100,100,100
"Harbourfront East, Union Station, Toronto Islands",100,100,100,100,100,100


In [91]:
# one hot encoding
downtown_onehot = pd.get_dummies(downtown_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
downtown_onehot['Neighborhood'] = downtown_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [downtown_onehot.columns[-1]] + list(downtown_onehot.columns[:-1])
downtown_onehot = downtown_onehot[fixed_columns]

# downtown_onehot.head()

In [92]:
downtown_grouped = downtown_onehot.groupby('Neighborhood').mean().reset_index()
downtown_grouped

,Neighborhood,Yoga Studio,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Women's Store
0,Berczy Park,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00,0.00,0.00,0.017544,0.000000,0.000000,0.000000,0.00
1,"CN Tower, King and Spadina, Railway Lands, Har...",0.000000,0.000000,0.058824,0.058824,0.058824,0.117647,0.176471,0.117647,0.000000,...,0.000000,0.000000,0.00,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.00
2,Central Bay Street,0.015625,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00,0.00,0.00,0.015625,0.000000,0.000000,0.015625,0.00
3,Christie,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.00
4,Church and Wellesley,0.026316,0.013158,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.013158,...,0.013158,0.013158,0.00,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.00
5,"Commerce Court, Victoria Hotel",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.040000,...,0.000000,0.000000,0.00,0.00,0.00,0.020000,0.000000,0.000000,0.010000,0.00
6,"First Canadian Place, Underground city",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.030000,...,0.010000,0.000000,0.00,0.00,0.01,0.010000,0.000000,0.000000,0.010000,0.00
7,"Garden District, Ryerson",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.020000,0.000000,0.01,0.00,0.00,0.000000,0.010000,0.010000,0.010000,0.00
8,"Harbourfront East, Union Station, Toronto Islands",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.010000,0.000000,0.00,0.00,0.01,0.010000,0.000000,0.000000,0.010000,0.00
9,"Kensington Market, Chinatown, Grange Park",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00,0.00,0.00,0.056338,0.000000,0.042254,0.014085,0.00


In [93]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [94]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
downtown_venues_sorted = pd.DataFrame(columns=columns)
downtown_venues_sorted['Neighborhood'] = downtown_grouped['Neighborhood']

for ind in np.arange(downtown_grouped.shape[0]):
    downtown_venues_sorted.iloc[ind, 1:] = return_most_common_venues(downtown_grouped.iloc[ind, :], num_top_venues)

# downtown_venues_sorted.head()

In [95]:
# set number of clusters
kclusters = 5

downtown_grouped_clustering = downtown_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(downtown_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 3, 4, 2, 1, 1, 1, 1, 1, 1], dtype=int32)

In [96]:
# add clustering labels
downtown_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

downtown_merged = df_downtown

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
downtown_merged = downtown_merged.join(downtown_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

downtown_merged.head() # check the last columns!

,PostalCode,Borough,Neighborhood,Lat,Long,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306,1,Coffee Shop,Cheese Shop,Bakery,Cocktail Bar,Café,Pub,Restaurant,Seafood Restaurant,Farmers Market,Beer Bar
1,M5V,Downtown Toronto,"CN Tower, King and Spadina, Railway Lands, Har...",43.628947,-79.394420,3,Airport Service,Airport Terminal,Airport Lounge,Harbor / Marina,Boat or Ferry,Rental Car Location,Boutique,Coffee Shop,Plane,Sculpture Garden
2,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383,4,Coffee Shop,Italian Restaurant,Sandwich Place,Café,Salad Place,Juice Bar,Bubble Tea Shop,Burger Joint,Japanese Restaurant,Thai Restaurant
3,M6G,Downtown Toronto,Christie,43.669542,-79.422564,2,Grocery Store,Café,Park,Baby Store,Candy Store,Coffee Shop,Athletics & Sports,Nightclub,Bank,Italian Restaurant
4,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160,1,Coffee Shop,Sushi Restaurant,Japanese Restaurant,Gay Bar,Restaurant,Yoga Studio,Café,Bubble Tea Shop,Pub,Mediterranean Restaurant


In [54]:
# create map
downtown_clusters = fm.Map(location=[t_lat, t_long], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(downtown_merged['Lat'], downtown_merged['Long'], downtown_merged['Neighborhood'], downtown_merged['Cluster Labels']):
    label = fm.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    fm.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(downtown_clusters)
       
downtown_clusters

In [59]:
#get desired neighborhood
choice = 'Central Toronto'
df_neighborhood = df_torontos[df_torontos['Borough']==choice].reset_index(drop=True)

df_venues = getNearbyVenues(names=df_neighborhood['Neighborhood'],
                                 latitudes=df_neighborhood['Lat'],
                                 longitudes=df_neighborhood['Long'])

Davisville
Davisville North
Forest Hill North & West, Forest Hill Road Park
Lawrence Park
Moore Park, Summerhill East
North Toronto West, Lawrence Park
Roselawn
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
The Annex, North Midtown, Yorkville


In [73]:
df_neighborhood.shape

(9, 5)

In [60]:
# one hot encoding
def make_onehot(df_venues):
    onehot = pd.get_dummies(df_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
    onehot['Neighborhood'] = df_venues['Neighborhood'] 

# move neighborhood column to the first column
    fixed_columns = [onehot.columns[-1]] + list(onehot.columns[:-1])
    onehot = onehot[fixed_columns]

    return onehot

In [66]:
def get_sorted(df_grouped):
    num_top_venues = 10

    indicators = ['st', 'nd', 'rd']

    # create columns according to number of top venues
    columns = ['Neighborhood']
    for ind in np.arange(num_top_venues):
        try:
            columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
        except:
            columns.append('{}th Most Common Venue'.format(ind+1))

    # create a new dataframe
    venues_sorted = pd.DataFrame(columns=columns)
    venues_sorted['Neighborhood'] = df_grouped['Neighborhood']

    for ind in np.arange(df_grouped.shape[0]):
        venues_sorted.iloc[ind, 1:] = return_most_common_venues(df_grouped.iloc[ind, :], num_top_venues)

    return venues_sorted

In [68]:
df_choice_hot = make_onehot(df_venues)
df_choice_hot.head()

,Neighborhood,American Restaurant,BBQ Joint,Bagel Shop,Bank,Breakfast Spot,Brewery,Burger Joint,Bus Line,Café,...,Supermarket,Sushi Restaurant,Swim School,Tennis Court,Thai Restaurant,Toy / Game Store,Trail,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Yoga Studio
0,Davisville,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Davisville,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
2,Davisville,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Davisville,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Davisville,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [69]:
df_choice_group = df_choice_hot.groupby('Neighborhood').mean().reset_index()
df_choice_group.head()

,Neighborhood,American Restaurant,BBQ Joint,Bagel Shop,Bank,Breakfast Spot,Brewery,Burger Joint,Bus Line,Café,...,Supermarket,Sushi Restaurant,Swim School,Tennis Court,Thai Restaurant,Toy / Game Store,Trail,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Yoga Studio
0,Davisville,0.0,0.0,0.0,0.0,0.000000,0.03125,0.0,0.0,0.0625,...,0.0,0.0625,0.0,0.000000,0.03125,0.03125,0.000000,0.0,0.0,0.0
1,Davisville North,0.0,0.0,0.0,0.0,0.111111,0.00000,0.0,0.0,0.0000,...,0.0,0.0000,0.0,0.000000,0.00000,0.00000,0.000000,0.0,0.0,0.0
2,"Forest Hill North & West, Forest Hill Road Park",0.0,0.0,0.0,0.0,0.000000,0.00000,0.0,0.0,0.0000,...,0.0,0.2500,0.0,0.000000,0.00000,0.00000,0.250000,0.0,0.0,0.0
3,Lawrence Park,0.0,0.0,0.0,0.0,0.000000,0.00000,0.0,0.2,0.0000,...,0.0,0.0000,0.2,0.000000,0.00000,0.00000,0.000000,0.0,0.0,0.0
4,"Moore Park, Summerhill East",0.0,0.0,0.0,0.0,0.000000,0.00000,0.0,0.0,0.0000,...,0.0,0.0000,0.0,0.333333,0.00000,0.00000,0.333333,0.0,0.0,0.0


In [77]:
df_choice_sorted = get_sorted(df_choice_group)
df_choice_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Davisville,Sandwich Place,Dessert Shop,Pizza Place,Italian Restaurant,Café,Sushi Restaurant,Gym,Coffee Shop,Indian Restaurant,Indoor Play Area
1,Davisville North,Hotel,Dance Studio,Food & Drink Shop,Park,Pizza Place,Sandwich Place,Department Store,Gym / Fitness Center,Breakfast Spot,Bus Line
2,"Forest Hill North & West, Forest Hill Road Park",Sushi Restaurant,Trail,Park,Jewelry Store,Fried Chicken Joint,Donut Shop,Farmers Market,Fast Food Restaurant,Food & Drink Shop,Yoga Studio
3,Lawrence Park,Jewelry Store,Park,Swim School,Dim Sum Restaurant,Bus Line,Yoga Studio,Food & Drink Shop,Donut Shop,Farmers Market,Fast Food Restaurant
4,"Moore Park, Summerhill East",Trail,Tennis Court,Park,Yoga Studio,Food & Drink Shop,Diner,Donut Shop,Farmers Market,Fast Food Restaurant,Garden
5,"North Toronto West, Lawrence Park",Coffee Shop,Clothing Store,Yoga Studio,Spa,Fast Food Restaurant,Italian Restaurant,Diner,Mexican Restaurant,Park,Pet Store
6,Roselawn,Home Service,Garden,Dim Sum Restaurant,History Museum,Gym / Fitness Center,Gym,Greek Restaurant,Gourmet Shop,Gas Station,Fried Chicken Joint
7,"Summerhill West, Rathnelly, South Hill, Forest...",Pub,Coffee Shop,American Restaurant,Sports Bar,Vietnamese Restaurant,Fried Chicken Joint,Light Rail Station,Liquor Store,Pizza Place,Restaurant
8,"The Annex, North Midtown, Yorkville",Café,Sandwich Place,Coffee Shop,Pub,Pharmacy,Pizza Place,Liquor Store,Donut Shop,Park,Middle Eastern Restaurant


In [97]:
# set number of clusters
kclusters = 5

grouped_clustering = df_choice_group.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(grouped_clustering)

# return kmeans

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 3, 4, 2, 0, 1, 0, 0], dtype=int32)

In [98]:
# add clustering labels
df_choice_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

df_merged = df_neighborhood

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
df_merged = df_merged.join(df_choice_sorted.set_index('Neighborhood'), on='Neighborhood')

df_merged.head() # check the last columns!

,PostalCode,Borough,Neighborhood,Lat,Long,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4S,Central Toronto,Davisville,43.704324,-79.388790,0,Sandwich Place,Dessert Shop,Pizza Place,Italian Restaurant,Café,Sushi Restaurant,Gym,Coffee Shop,Indian Restaurant,Indoor Play Area
1,M4P,Central Toronto,Davisville North,43.712751,-79.390197,0,Hotel,Dance Studio,Food & Drink Shop,Park,Pizza Place,Sandwich Place,Department Store,Gym / Fitness Center,Breakfast Spot,Bus Line
2,M5P,Central Toronto,"Forest Hill North & West, Forest Hill Road Park",43.696948,-79.411307,3,Sushi Restaurant,Trail,Park,Jewelry Store,Fried Chicken Joint,Donut Shop,Farmers Market,Fast Food Restaurant,Food & Drink Shop,Yoga Studio
3,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,4,Jewelry Store,Park,Swim School,Dim Sum Restaurant,Bus Line,Yoga Studio,Food & Drink Shop,Donut Shop,Farmers Market,Fast Food Restaurant
4,M4T,Central Toronto,"Moore Park, Summerhill East",43.689574,-79.383160,2,Trail,Tennis Court,Park,Yoga Studio,Food & Drink Shop,Diner,Donut Shop,Farmers Market,Fast Food Restaurant,Garden


In [100]:
# create map
# downtown_clusters = fm.Map(location=[t_lat, t_long], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(df_merged['Lat'], df_merged['Long'], df_merged['Neighborhood'], df_merged['Cluster Labels']):
    label = fm.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    fm.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(downtown_clusters)
       
downtown_clusters